In [1]:
import pandas as pd
import jieba
import jieba.analyse
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

pd.set_option('display.max_colwidth', None)#setting the maximize string show

In [2]:
df = pd.read_csv("KCC Data/NewsCisDepAll_Seg-35325.csv", encoding='utf-8-sig') 

In [3]:
df.head(2)

,編號,質詢單位,詳細,逐字稿
0,15601,陳美雅,2-1-1040504-2-陳美雅-1,陳議員美雅：本席的市政總質詢想針對高雄市民關心的許多議題就教市府團隊。首先本席要請教市長，高雄市目前對於大陸之間的兩岸關係，市長的態度如何，認為我們跟大陸之間的交流，你是支持還是反對？請市長表達你的立場。主席（康議長裕成）：市長，請回答。陳市長菊：高雄市是一個開放的城市，高雄市跟中國大陸之間，我們一向都是鼓勵交流。同時我個人為了2009年世界運動會、2013年亞太城市高峰會議，我都曾經到中國大陸相關的城市邀請他們來參與。我想做為一個開放的城市，我們都是…。陳議員美雅：所以市長你認為台灣目前應該維持什麼樣的政治立場，你目前的政治立場可不可以更簡單一點？陳市長菊：台灣現階段就是維持現狀。陳議員美雅：我們的現狀可不可以請市長用更白話一點告訴市民？陳市長菊：這跟市政無關，我想今天不討論政治議題。陳議員美雅：本席現在要了解的，是要跟高雄市民講，如果剛剛市長表達的立場是未來也希望增加兩岸之間的交流…。陳市長菊：我同意。陳議員美雅：那麼市長是否可以更坦然的面對這個問題，讓兩岸之間的交流和平發展？陳市長菊：當然。陳議員美雅：所以你認為目前台灣跟大陸之間朝著所謂的和平發展，是指九二共識你也認同的意思嗎？陳市長菊：兩岸之間朝著更進步的的發展，我想九二共識有不同的解讀，這個部分在台灣社會有不同的看法…。陳議員美雅：市長你的解讀呢？陳市長菊：我想這跟今天的市政無關。謝謝。陳議員美雅：這跟未來高雄市政，大陸未來如何跟高雄市之間做的觀光文化交流，甚至未來有沒有任何經貿交流有關，市長你好像一直迴避這個問題。陳市長菊：我們都贊成跟中國大陸之間不斷的開放交流，藉由民間更多的交流，異中求同，然後爭取更多彼此的認知及認識，這樣兩岸之間才能更趨和緩。謝謝。陳議員美雅：謝謝市長，市長剛剛的意思也尊重所謂的九二共識。陳市長菊：沒有，那是你的解讀，我不是這樣解讀的。陳議員美雅：市長，你的解讀讓你更白話的說一句話，你就用一句話表達你的立場，我的市政質詢時間非常寶貴。陳市長菊：我的答覆也是很寶貴，時間是共同的。陳議員美雅：所以你的立場到底是什麼呢？陳市長菊：我的立場很清楚，兩岸之間異中求同，兩岸之間有很多歷史發展的過程不一樣，兩岸之間應該要更多的往來、互相尊重、異中求同。謝謝。陳議員美雅：互相尊重嘛！謝謝你認同所謂的互相尊重這一點。我想高雄不要再有所謂的意識形態，議員在高雄市議會要為高雄市、為台灣表達一個立場，我們希望讓高雄市民能夠生活得更好，不要總是赤字困難。我們希望能創造高雄跟兩岸之間，甚至各國之間是友好的環境，讓大家樂於來高雄市投資，我們就是希望讓高雄市民未來有更多的就業機會，未來高雄市能自創更多的自有財源。這是市長未來應該要為高雄市民奮鬥努力的。
1,15602,陳美雅,2-1-1040504-2-陳美雅-10,陳議員美雅：請尊重本席的時間。市長，這就是我們的官員，新的局長也要來和議員嘔，難怪副市長做錯誤的示範，各位高雄市民，民進黨完全執政之下，居然就是這樣蠻橫的狀態，對於議員的質詢不針對問題回答，還要用回嗆的方式，甚至沒有請他回答時，他也要在台下用叫囂的方式。本席再請教市長，45億6


In [4]:
df['質詢單位'].unique()

array(['陳美雅', '李喬如', '簡煥宗', '蔡金晏', '國民黨團', '民進黨團'], dtype=object)

In [5]:
ly = df[df.質詢單位 == "蔡金晏"]

In [6]:
doc_names = range(len(ly)) #有幾篇文章
doc_names

range(0, 125)

In [7]:
text_list = list(ly['逐字稿'])
ly_docs = pd.DataFrame(columns=['jieba_results'])
ly_docs['jieba_results'] = ly_docs['jieba_results'].astype('str')

In [8]:
words = jieba.lcut(text_list[1], cut_all = False)
print(words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.661 seconds.
Prefix dict has been built successfully.


['再來', '回到', '我', '這次', '總質', '詢要', '討論', '的', '題目', '，', '大概', '就', '幾個', '問題', '。', '第一', '個', '是', '產業', '的', '現況', '，', '左邊', '這張', '圖應', '該', '有', '很多', '人', '看過', '，', '右上角', '是', '老舊', '建物', '的', '保存', '；', '右下角', '則是', '再生', '水', '的', '利用', '，', '上星期', '行政院', '也', '通過', '再生', '水', '草案', '，', '這已經', '不是', '第一次', '提到', '，', '高雄市', '推行', '再生', '水', '，', '是', '在', '103', '年', '李鴻源', '李老師', '當內', '政部長', '時', '，', '就', '協助', '高雄市', '將鳳山', '污水', '處理廠', '做', '了', '改造', '，', '新聞稿', '好像', '是', '106', '年會', '完成', '，', '接下', '來', '我們', '可以', '再', '討論', '。', '去年', '731', '氣爆', '，', '我', '想', '大家', '都', '很', '不忍心', '看到', '，', '我們', '也', '針對', '高雄市', '既有', '的', '工業', '管線', '制定', '了', '自治', '條例', '要', '來', '管制', '，', '自治', '條例裡', '有', '很多', '都', '是', '我們', '一致', '訴求', '，', '包括', '要', '有', '確實', '、', '安全', '、', '謹慎', '、', '定期', '的', '相關', '維管', '報告', '及維管', '工作', '。', '國民黨', '黨團', '成員', '的', '意見', '、', '分歧', '的', '地方', '，', '其實', '我們', '的', '目標', '是', '一致', '的', '，', '只是', '有些', '細部', '不同', '。'

In [9]:
punctuation = " //，：:""()\n!！？｡＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
re_punctuation = "[{}] ".format(punctuation)

#creat a df to store the jieba result
ly_docs = pd.DataFrame(columns=['jieba_results']) # 建立df
ly_docs['jieba_results'] = ly_docs['jieba_results'].astype('str')

jieba.set_dictionary("KCC Data/Dict/dict.big.txt") #更改預設辭典（dict.big為繁體版）
jieba.load_userdict("KCC Data/Dict/KccDict2020.txt") #領域辭典
jieba.analyse.set_stop_words("KCC Data/Dict/KccStopWord2020.txt")

stopword_list = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords: #斷詞辭典
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

for i in doc_names: #從第一篇到最後一篇
    words = jieba.cut(text_list[i], cut_all = False)
    text = ''
    for word in words:
        text = text + ' ' + str(word) 
    #print(words)
    #print(i) #第幾篇文章
    #print(text) #第幾篇文章的斷詞結果
    text = re.sub(re_punctuation, "", text) # 把上面那些符號從text中刪掉
    text = re.sub(r'[0-9]','',text) # 刪掉數字
    text = re.sub(r'[a-zA-Z]','',text) #刪掉英文字
#刪除文字 更多標點符號
    text = re.sub(r'[-<>#。！⋯.➡?=&▶_%♀!❗🎉⏰💪㊙️✨🔺😁😅😍⬇🙏✅🎈🔥📱👍👊🔸📍👉📢👩📌💼]','',text)  # 注意裡面不能有空白！
    s = pd.Series({'jieba_results': text})
    ly_docs = ly_docs.append(s, ignore_index=True)#每次的斷詞結果都加回liu_docs

Building prefix dict from C:\Users\User\OneDrive\POLITICS-AND-INFORMATION-master\20210517_code\KCC Data\Dict\dict.big.txt ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ue06d503e3529cb2586146b6160a5167a.cache
Loading model cost 1.141 seconds.
Prefix dict has been built successfully.


In [10]:
ly_docs

,jieba_results
0,蔡議員金晏 我 剛剛 到 議會 所以 前 兩位 同仁 的 質詢 我 不 確定 有沒有 提到 就是 上星期 發生 備受 全國 矚目 的 社會 案件 凸顯 了 一個 議題 這 幾年 高雄市 也 是 我們 都 強調 校園 和 社區 要 融合 因此 校園 的 人本 環境 很多 都 把 圍牆 拆掉 了 上星期 的 這個 案件 似乎 也 凸顯 了 這樣 的 問題 包括 縣市合併 以後 我們 知道 市政府 財政困難 很多 學校 也許 白天 還有 一些 警衛 人力 但是 到 了 晚上 因為 經費 的 問題 很多 都 改成 所謂 的 機械式 保全 在 這樣 的 氛圍 之下 如果 要 把 圍牆 再 重新 蓋起來 也許 有點 因噎廢食 我 相信 大家 及 教育局 都 很 關注 這些 議題 也 希望 我們 的 學生 尤其 是 國小 國中 和 國小 還是 有 體格 上 或 應變 上 的 差異 是不是 在 國小 部分 因為 今天 一 早就 有 某間 學校 的 家長 會長 打電話 給我 提到 這樣 的 議題 因為 是 全國 矚目 的 案件 是不是 請 教育局 說明 我 相信 目前 貴局 應該 也 在 思考 要 如何 來 因應 這樣 的 問題 當然 發生 的 原因 是 一個 社會 問題 這個 社會 問題 似乎 沒 辦法 找到 正確 的 解答 可是 至少 在 校園安全 的 部分 我們 要 如何 來 做好 是不是 請 局長 簡單 答覆 主席 康議長裕成 請 局長 回答 教育局范局長巽綠 有關 這件 校園 的 不幸 事件 市政府 整個 團隊 包括 警察局 教育局 衛生局 都 全部 通力合作 檢視 整個 校園 防護 安全 網路 的 聯 繫 機制 學校 該 做 什麼 檢視 全部 趁 這個 事件 的 發生 再 做 一次 清點 今天 早上 各級 學校 全部 都 要 召開 自己 的 學校 安全 會議 找出 所有 的 漏洞 及 他們 不 放心 的 地方 下午 教育局 和 警察局 又 會 聯合 再 找 學校 的 代表 重新 再 做 一次 檢視 務必 讓 大家 都 能夠 提高 警覺 在 一個 良好 的 安全 防護網 之下 讓 學童 可以 安心 上學 讓 家長 也 能夠 安心 把 學生 送到 學校 這部分 需要 整個 社區 的 力量 也 要 巡守隊 的 力量 家長 保全 及 警察局 整體 都動 起來 所以 我們 不會 想到 要 不要 馬上 把 圍牆 加高 其實 這是 一件 無 預警 的 隨機 殺人 事件 是 現在 新型 態 的 犯罪 手法 整個 社會 都 要 提高 警覺 蔡議員金晏 局長 提到 已經 開始 在 因應 目前 還是 針對 整個 防護網 的 建置 我 想 防護網 再 怎麼 建置 嚴密 針對 這種 故意 犯 或 蓄意 侵入 校園 的 罪犯 似乎 很難 在 教導 的 部分 是 可以 努力 的 地方 但是 小學生 的 心智 成熟度 畢竟 還 停留 在 成長 的 階段 過程 中 要 把 他 教導 成 如果 遇到 這樣 的 狀況 能夠 多有 應變能力 其實 很 有限 我 具體 的 問過 家長 會長 他 的 訴求 也 和 我 的 訴求 一樣 要 把 圍牆 重新 蓋起來 不太可能 第一 經費 就 有 問題 是不是 要 走 這樣 的 回頭路 也 有 很大 的 討論 空間 具體 的 一點 是 是不是 能夠 去 評估 看看 提出 這樣 的 想法 可能 有 很多 人會 反對 我 希望 教育局 可以 去 評估 整個 校園 開放 管制 的 時間 和 空間 要 如何 去 調配 我 想 社區 融入 校園 是 一個 趨勢 但是 遇到 這樣 的 事件 不免 讓 我們 想到 目前 校園 開放 的 狀態 下課時間 人 那麼 多 巡守隊 也好 警力 也好 有 辦法 嗎 很難 他 需要 半夜 跑 進去 躲起來 這也 很 難 我們 不去 預設 任何 立場 希望 可以 把 它 做到 最 完善 去 評估 在 時間 及 空間 上 是不是 需要 更 嚴格 的 管制 還是 怎樣 尤其 是 針對 國小 的 部分 我 希望 教育局 能夠 針對 這個 大家 非常 關注 的 問題 可否 在 一個 星期 內能 有 初步 的 評估 結果 這方面 是否 可行 或 還有 更 多 的 討論 空間 我 希望 帶給 這些 小朋友 安全 家長 安心 的 學習 環境 我們 一定 也 接到 很多 電話 比如 小朋友 不敢 上學 等等 的 問題 這是 很 重要 的 一個 議題 所以 我 把 它 移到 我 要 探討 的 議題 之前 先 向 市府 團隊 就 教
1,再 來 回到 我 這次 總質詢 要 討論 的 題目 大概 就 幾個 問題 第一 個 是 產業 的 現況 左邊 這張 圖 應該 有 很多 人 看過 右上角 是 老舊 建物 的 保存 右下角 則是 再生 水 的 利用 上星期 行政院 也 通過 再生 水 草案 這 已經 不是 第一次 提到 高雄市 推行 再生 水 是 在 年 李鴻源 李老師 當 內政部長 時 就 協助 高雄市 將 鳳山污水處理廠 做 了 改造 新聞稿 好像 是 年會 完成 接下來 我們 可以 再 討論 去年 氣爆 我 想 大家 都 很 不忍心 看到 我們 也 針對 高雄市 既有 的 工業管線 制定 了 自治條例 要 來 管制 自治條例 裡有 很多 都 是 我們 一致 訴求 包括 要 有 確實 安全 謹慎 定期 的 相關 維管 報告 及維管 工作 國民黨 黨團 成員 的 意見 分歧 的 地方 其實 我們 的 目標 是 一致 的 只是 有些 細部 不同 其實 整個 問題 我 覺得 最 重要 的 是 這張 圖 圈起來 的 地方 就是 去年 發生意外 的 管線 我們 可以 看到 它 是 吋 管 從福 聚到 石化 站 石化 站 應該 在 前鎮 我們 可以 看到 從 當時 中油 提出 的 好幾個 工程 有 總廠 到 石化 站 總廠 到苓 雅站 苓 雅站 到 大站 等等 我們 就 可以 知道 這些 管線 到底 在 哪裡 就 在 市區 可能 跟 原來 的 因為 這個 大概 是 民國 七 八十年 完工 的 所以 它 有 遠離 原來 的 新興 前金 苓 雅 鹽埕 鼓山 但是 就 在 這個 外圍 但是 這 十幾 二十年 的 城市 發展 剛剛 講的 這些 也許 在 那 時候 是 相對 比較 邊緣 的 地方 但是 現在 那裡 也 已經 變成 市中心 了 變成 人口稠密 的 住宅區 我們 可以 知道 很多 石化 原料 都 是 依賴 進口 進口 都 走 海運 我們 的 石化 廠 很多 都 設在 仁大工業區 勢必 要 拉 管線 或是 透過 槽車 運送 在 審議 自治條例 的 時候 局長 也 有 說 槽車 的 不穩定性 上 這個 都 有 它 的 優劣 我想問 市長 如果 你 要 澈底 解決 這個 問題 的話 有 兩點 第一 慢慢 淘汰 石化 產業 這個 有 可能 嗎 石化 專區 會 設在 哪裡 我 不 知道 但是 石化 專區 有沒有 必要 設置 這要 請教 陳市長 因為 陳市長 是 市政 的 龍頭 這個 很 重要 如果 我們 要 設置 石化 專區 未來 應該 怎麼 做 請 陳市長 回答 我們 是 要 石化 專區 還是 要將 石化 產業 慢慢 淘汰 掉 主席 康議長裕成 請 市長 回答 陳市長菊 今天 我們 所 討論 的 石化 管線 是 一個 嚴肅 的 問題 我們 不能 讓 高雄 市民 長期 活在 恐懼 當中 我 不 知道 我 的 不 知道 過去 我們 就是 不 知道 我們 自己 的 不 知道 今天 高雄市 在 產業 的 布局 所有 的 石化 污染 都 在 高雄 現在 在 討論 過去 的 歷史 時 我們 都 是 用 包容 來看 整體 的 發展 如果 今天 對 環保 安全 的 議題 大家 都 一樣 重視 這個 沒有 政黨 的 問題 每 一位 市民 都 是 不同 政黨 的 支持者 蔡議員金晏 現在 已經 是 一個 普世 的 價值 現在 的 潮流 陳市長菊 對 我們 現在 就是 一個 普世 的 價值 今天 所有 的 產業 都 必須 建立 在 安全 的 基礎 上 這個 產業 的 發展 才 有 意義 今天 的 石化業 高雄市政府 是 接納 理解 他們 的 現狀 但是 我們 必須 合作 這個 合作 就是 產業 現階段 還要 繼續 生存 的話 至少 要 讓 高雄 市民 感覺 到 是 安全 的 所有 合理 嚴格 的 監督 及 管理 我 認為 業者 必須 要 和 我們 合作 這個 產業 才 有 辦法 在 高雄 生存 但是 產業 要 不斷 的 升級 如果 沒有 升級 繼續 污染 的話 每天 民眾 都 有 不同 的 抗爭 及 抗議 市民 的 健康 受到 很大 的 威脅 所以 你問 有沒有 必要 成立 石化 專區 這個 問題 在 二 三十年 前 都 有 在 討論 但是 一直 沒有 具體 像 新加坡 的 石化 專區 不 影響 他們 蔡議員金晏 在 一個 島上 陳市長菊 對 不會 影響 到 整個 城市 的 安全 及 發展 另外 一方面 也給 石化業 者 安心 經濟部 在 何美玥 擔任 部長 時 就 提到 我們 應該 要 有 一個 石化 專區 長期 而言 石化業 者 的 生存 要 有 石化 專區 必須 遠離 市民 生活圈 這個 方向 是 正確 的 蔡議員金晏 所以 市長 也 同意 這個 方向 陳市長菊 這個 方向 要 有 石化 專區 但 石化 專區 要 設置 在 哪裡 石化 專區 經濟部 的 規劃 為何 到 現在 他們 都還 沒有 來 和 我們 討論 高雄 市民 要求 安全 沒有 污染 市

In [11]:
ly_docs.to_csv('蔡金晏_output.txt')

In [12]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [13]:
fList = []
with open("蔡金晏_output.txt","r", encoding='utf-8-sig') as files:
     for file in files:
        #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
        fList.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
print("fList[:1] = ", fList[:1])
print("------------------------------")

fList[:1] =  [['jieba_results']]
------------------------------


In [14]:
dict = corpora.Dictionary(fList)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(8661 unique tokens: ['jieba_results', '一些', '一件', '一個', '一定']...)
------------------------------


In [15]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in fList]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('jieba_results', 1)


In [16]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("蔡金晏_ldaoutput.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(fList), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.005*"房市" + 0.005*"公告地價" + 0.004*"政策" + 0.004*"交易量" + 0.004*"開源節流" + 0.004*"地價稅" + 0.003*"景氣" + 0.003*"房價" + 0.003*"每戶" + 0.003*"增稅" + 0.003*"流水" + 0.002*"所得" + 0.002*"再生" + 0.002*"稅收" + 0.002*"房地" + 0.002*"消費市場" + 0.002*"抑制" + 0.002*"會漲" + 0.002*"房地產" + 0.002*"合一"

strTM =  10.009*"北漂" + 0.006*"漁民" + 0.003*"海造陸" + 0.003*"大林煉油廠" + 0.003*"重工業" + 0.003*"洲際" + 0.002*"出生" + 0.002*"竹筏" + 0.002*"專區" + 0.001*"連任" + 0.001*"主計" + 0.001*"鋼鐵" + 0.001*"高值" + 0.001*"儲槽" + 0.001*"洲際貨櫃中心" + 0.001*"南星計畫" + 0.001*"集結" + 0.001*"漁權" + 0.001*"旅高" + 0.001*"南漂"

strTM =  20.016*"道路" + 0.015*"都市計畫" + 0.013*"房子" + 0.012*"巷道" + 0.010*"既成" + 0.009*"智慧" + 0.008*"資料" + 0.007*"城市" + 0.007*"票證" + 0.006*"指定" + 0.006*"地政" + 0.006*"建築" + 0.006*"建管" + 0.006*"可以" + 0.006*"現有" + 0.006*"內惟路" + 0.006*"整合" + 0.006*"電子" + 0.006*"個案" + 0.005*"資訊"

strTM =  30.011*"這個" + 0.011*"保存" + 0.010*"蔡議員金晏" + 0.009*"文化局" + 0.008*"文化局尹局長立" + 0.008*"這些" + 0.008*"我們" + 0.007*"歷史" + 0.007*"市長" + 0.006*"知道" + 0.006*"因為" + 0.006*"文

In [17]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

pyLDAvis.enable_notebook
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict, doc_topic_dist=None)
"""
参数说明：
prepare(topic_model, corpus, dictionary, doc_topic_dist=None)
topic_model：训练得到的 gensim LdaModel 对象，不支持其他 gensim 模型类型
corpus：以词袋形式表示的语料（用于训练主题模型的语料）
dictionary：用于构建词袋模型的字典，即 gensim Dictionary 对象
doc_topic_dist：可选参数，用于传入 LDA 模型的文档主题分布，默认取值为 None，当需要多次调用 prepare 函数时，可以传入该参数
"""
pyLDAvis.display(vis)